<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">Introduction</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>
    This notebook presents and discusses how to encode the Weierstrass elliptic functions, namely:
<ul>
    <li>$\wp(z) = \wp(z, g_2, g_3)$ – the Weierstrass $\wp$ elliptic function for argument $z$, characterized by the lattice invariants $g_2$ and $g_3$.</li>
     <li>$\wp'(z) = \wp'(z, g_2, g_3)$ – first defivative of the Weierstrass $\wp$ elliptic function for argument $z$, characterized by the lattice invariants $g_2$ and $g_3$.</li>
    <li>$\sigma(z) = \sigma(z, g_2, g_3)$ – the Weierstrass $\sigma$ function for argument $z$, characterized by the lattice invariants $g_2$ and $g_3$.</li>
    <li>$\zeta(z) = \zeta(z, g_2, g_3)$ – the Weierstrass $\zeta$ function for argument $z$, characterized by the lattice invariants $g_2$ and $g_3$.</li>
</ul>

Additionally, the inverse function $\wp^{-1}$ of the Weierstrass $\wp$ function is introduced. The notebook also defines other useful quantities such as:
<ul>
    <li>$\omega_i = \omega_i (g_2, g_3)$ – the half-periods (or lattice generators) characterised by the lattice invariants $g_2$ and $g_3$.</li>
    <li>$g_2, g_3$ – the lattice invariants expressed as functions of the half-periods $\omega_1$ and $\omega_3$.</li>
    <li>Reduced argument $z$ to the fundamental domain</li>
</ul>

The functions presented here are equivalent to those available in Wolfram Mathematica 14 in the field of Weierstrass elliptic functions.

The following two libraries are required to run the code:
- <code style="color: rgb(81,206,9); font-size: 14px;">mpmath</code> 
- <code style="color: rgb(81,206,9); font-size: 14px;">numpy</code> 

The code is primarily based on definitions from the <a href="https://dlmf.nist.gov/23" target="_blank">DLMF</a> database but also references the <a href="https://functions.wolfram.com/EllipticFunctions" target="_blank">Wolfram</a> database and <a href="https://en.wikipedia.org/wiki/Carlson_symmetric_form" target="_blank">Wikipedia</a>.

### **Author:**
This notebook was created by Adam Cieślik.

### **Python and Sage Libraries:**
Based on this notebook, dedicated **Python** and **SageMath** libraries have been developed to facilitate the computation of Weierstrass elliptic functions.

</p>
</div>

In [2]:
from mpmath import *
import numpy as np

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
We set two important parameters:

  <code style="color:  rgb(81,206,9);font-size: 14px;">mp.dps = 25</code>: Specifies the precision of calculations to 25 decimal digits. By default, the precision is set to 15 digits, but you can adjust it to your needs.</li>

  <code style="color:  rgb(81,206,9);font-size: 14px;">mp.pretty = True</code>: Enables pretty formatting of results. When this option is active, mpmath displays results in a more readable way, e.g. by rounding the numbers and using mathematical notation.
</ul>    
</div>    

In [3]:
mp.dps = 25
mp.pretty = True

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">1. $\wp$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on formulas <a href="http://dlmf.nist.gov/23.6.E1" target="_blank">(23.6.1)</a> and <a href="http://dlmf.nist.gov/23.6.E5" target="_blank">(23.6.5)</a> we can write
$$
    \wp (z) = \left( \frac{\pi}{2\omega_1} \theta_3(0,q)\theta_4(0,q) \frac{\theta_2(\pi z/(2\omega_1),q)}{\theta_1(\pi z/(2\omega_1),q)}  \right)^2 + \frac{\pi^2}{12 \omega_1^2} \left( \theta_2^4(0,q) + 2\theta_4^4(0,q)\right),
$$
where $\theta_i$ are Jacobi theta functions, $\omega_i$ are half-periods, $q=e^{i\pi\tau}$, $\tau=\omega_3/\omega_1$, and 
$$
     \operatorname{Im}(\tau)>0
$$

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.1. $\omega_j$ </h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
    
Based on formulas <a href="http://dlmf.nist.gov/23.6.E16" target="_blank">(23.6.16-17)</a> for a Legendre elliptic integral of the first kind, we obtain:
    
$$
    {K}^{2}=(K\left(k\right))^{2}=\omega_{1}^{2}(e_{1}-e_{3}),
$$
    
$$
    {K}'^{2}=(K\left(k'\right))^{2}=\omega_{3}^{2}(e_{3}-e_{1}),
$$
where 
$$
    k^2 = \frac{e_2 - e_3}{e_1 - e_3}, \quad k'^2=\frac{e_1-e_2}{e_1-e_3}.
$$
Thus, the half-periods are given by:
$$
    \omega_1 = \pm \frac{1}{\sqrt{e_1-e_3}} K(k), \quad \omega_3 = \pm \frac{1}{\sqrt{e_3 - e_1}}K(k').
$$


TThe choice of the appropriate sign for the half-periods is not strictly defined. This arises directly from their definition: 
$$
    \wp(z) = \wp(z + 2m \omega_1 + 2n\omega_3),
$$
where $m,n\in N$. It is important that the condition
$$
     \operatorname{Im}(\tau)>0
$$
is satisfied.
Thus, we make use of this condition in the definition of $\omega_1$, so that we do not have to worry about it in $\omega_3$. To obtain $\omega_2$ we simply use the relation <a href="http://dlmf.nist.gov/23.2.E1" target="_blank">(23.2.1)</a>:
$$
    \omega_1 + \omega_2 + \omega_3 = 0.
$$

</div>

In [4]:
def Omega1(g2, g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Calculation of parameters k^2 and k'^2
    ksqr = (e2 - e3) / (e1 - e3)
    ksqrp = (e1 - e2) / (e1 - e3)


    # Calculating omega1 and omega3
    omega1 = (1 / sqrt(e1 - e3)) * ellipk(ksqr)
    omega3 = (1 / sqrt(e3 - e1)) * ellipk( ksqrp)

    # Calculation of the Im(tau)
    Imtau = (omega3 / omega1).imag

    # Condition for selecting omega1 sign
    res = omega1 if Imtau > 0 else -omega1

    return res

In [5]:
def Omega3(g2,g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots

    # Calculation of parameter k'^2
    ksqrp = (e1-e2)/(e1-e3)

    # Calculating omega3
    omega3= (1/sqrt(e3-e1))*ellipk(ksqrp)

    return omega3

In [6]:
def Omega2(g2,g3):
    res = -Omega1(g2,g3) - Omega3(g2,g3)
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.2. $g_2$, $g_3$ </h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

The invariants can be calculated using the formula provided by   <a href="https://functions.wolfram.com/EllipticFunctions/WeierstrassInvariants/27/ShowAll.html" target="_blank">Wolfram</a>
$$
    g_2 = \frac{1}{12} \left( \frac{\pi}{\omega_1} \right)^4 \left[ \theta_2(0,q)^8 - \theta_3(0,q)^4 \theta_2(0,q)^4 + \theta_3(0,q)^8 \right] ,
$$
$$
    g_3 = \left( \frac{\pi}{2 \omega_1} \right)^6 \left[ \frac{8}{27} \left( \theta_2(0,q)^{12} + \theta_3(0,q)^{12} \right) - \frac{4}{9}\left( \theta_2(0,q)^4 + \theta_3(0,q)^4\right) \theta_2(0,q)^4 \theta_3(0,q)^4 \right].
$$
</div>

In [7]:
def WeierstrassInvariants(omega1, omega3):
    ttau = omega3 / omega1
    if ttau.imag<0:
        print("Im(tau)<0, change Omega1 or Omega3")
        
    q = qfrom(tau=ttau)

    # Calculation of g2 and g3
    g2 = (pi**4 / (12 * omega1**4)) * (jtheta(2, 0, q)**8 - jtheta(3, 0, q)**4 * jtheta(2, 0, q)**4 + jtheta(3, 0, q)**8)
    
    g3 = (pi / (2 * omega1))**6 * ((8/27)*(jtheta(2, 0, q)**12 + jtheta(3, 0, q)**12)-(4/9)*(jtheta(2, 0, q)**4 +jtheta(3, 0, q)**4) * jtheta(2, 0, q)**4 * jtheta(3, 0, q)**4)

    return (g2, g3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.3
    . $\wp$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now, using the encoded half-periods and the Jacobi theta functions encoded in the <code style="color:  rgb(81,206,9);font-size: 14px;">mpmath</code> package, one can present a code for the Weierstrass elliptic function.
</div>

In [8]:
def WeierstrassP(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    FirstCase = (pi/(2*Omega1(g2,g3))) *jtheta(3, 0,q) *jtheta(4, 0, q) * jtheta(2, pi*z/(2*Omega1(g2,g3)),q)/jtheta(1, pi*z/(2*Omega1(g2,g3)),q)
    SecondCase = (1/12)*(pi/Omega1(g2,g3))**2 * (jtheta(2, 0,q)**4 + 2*jtheta(4, 0,q)**4)
    res = FirstCase**2 + SecondCase
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.4. Fundamental region</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<p>

For elliptical functions, we can present all arguments as
$$
    z = z(\mathbf{\varepsilon}) = \omega_1 \varepsilon_1 + \omega_3 \varepsilon_2.
$$
where $\varepsilon_1$, $\varepsilon_2$, are called characteristics and $\mathbf{\varepsilon}=(\varepsilon_1,\varepsilon_2)$ is a characteristic vector. The region spanned by the characteristic vector, for which $\varepsilon_1$, $\varepsilon_2 \in [0,2)$ holds, is called the fundamental domain (period-parallelograms,or meshes).

From the periodicity condition
$$
    \wp(z) = \wp(z+2\omega_1 n + 2\omega_3 m)
$$
we see that, in order to obtain the appropriate value of the function, we may shift its argument. This can have numerical significance because our definition of the function $\wp$ is based on the $\theta_i$ function, which may be defined by a series. Therefore, the smaller the arguments, the faster the convergence of the series and, consequently, the easier it is to compute the correct value.

In addition, when we calculate the inverse function of $\wp^{-1}$, it is important to have a good "point of reference" against which we will compare the results.

To convert the given argument $\tilde{z}$ to its counterpart $z$ in the fundamental domain, we need to proceed as follows. First, note that, by setting
$$
    \frac{1}{2}\varepsilon_1 = \nu, \qquad \frac{1}{2}\varepsilon_2 = \mu,
$$
we have
$$
    z = 2\omega_1\left( \mu + \nu\tau \right),
$$
where $\mu$, $\nu \in \mathbb{R}$ are defined uniquely modulo $\mathbb{Z}$. For a given  $\tilde{z}$, we want to find the appropriate integers   $n_1$ and $n_2$ such that
$$
    z = \tilde{z} - 2\omega_1 n - 2\omega_3 m.
$$
From the above equations we can easily deduce that
$$
    c = \omega_1^{-1}\tilde{z} = 2\mu+ 2n + ( \nu + 2m)\tau.
$$
In this way, $c$ is a complex number that already contains information about the coordinates $\mathbf{\mu}$ and $\mathbf{\nu}$ (although not yet separated into real and imaginary parts). Let us write this in a simplified form:
$$
    \mathbf{c} = \omega_1^{-1}\tilde{z} = \alpha + \beta\tau,
$$
where
$$
    \alpha = 2(\mu + n), \qquad \beta = 2(\nu + m).
$$
We know that $\tau$ has a positive imaginary part; therefore, to separate $c$ into its real and imaginary components, we can use the fact that the imaginary part depends only on $\beta$. In particular,
$$
    \operatorname{Im}(c) = \operatorname{Im}(\tau)\beta,
$$
so that
$$
    \beta = \bigl[\operatorname{Im}(\tau)\bigr]^{-1}\operatorname{Im}(c).
$$
Then, having determined $\beta$, we calculate
$$
    \alpha = \operatorname{Re}(c) - \operatorname{Re}(\tau)\mathbf{\beta}.
$$

Finally, having calculated the vectors $\mathbf{\alpha}$ and $\mathbf{\beta}$, we determine $\mathbf{\mu}$ and $\mathbf{\nu}$ by selecting the integer numbers $m$ and $n$ such that the resulting numbers lie in the range $[0,1)$. That is,
$$
    \mu = \frac{\alpha}{2} - n = \frac{\alpha}{2} - \lfloor \alpha/2\rfloor,
$$
$$
    \nu = \frac{\beta}{2} - m = \frac{\beta}{2} - \lfloor \beta/2 \rfloor.
$$

Now we can encode the function that brings the argument $\tilde{z}$ to the fundamental domain.
</p>
</div>

In [9]:
def reduce_argument(tilde_z, g2,g3 ):
    
    oomega1 = Omega1(g2, g3) 
    oomega3 = Omega3(g2,g3)
    ttau = oomega3/oomega1 
    # Step 1: Convert tilde_z to c coordinates: c = omega^{-1} * tilde_z
    c = (1/oomega1) * tilde_z 
    
    # Step 2: Separating c into its imaginary part to find beta
    Im_tau = ttau.imag
    Im_c = c.imag
    beta = (1/Im_tau) * Im_c  # beta ∈ R

    # Step 3: Calculation of alpha
    Re_tau = ttau.real
    Re_c =  c.real
    alpha = Re_c - Re_tau * beta  # alpha ∈ R
    
    # Step 4: Reducing the coordinates of mu and nu to the interval [0,1)
    mu = alpha/2- floor(alpha/2)
    nu = beta/2 - floor(beta/2)

    # Step 5: Reconstruction of the reduced z
    z_red =2* oomega1 * (mu + ttau * nu)
    
    return z_red

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.5. $\wp^{-1}$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Based on the formula <a href="http://dlmf.nist.gov/23.6.E30" target="_blank">(23.6.30)</a> for $t=\wp(z)$, we can write:
$$
    \wp^{-1}(t)=z = \frac{1}{2} \int^\infty_t \frac{du}{\sqrt{(u-e_1)(u-e_2)(u-e_3)}}.
$$
Now, we can use the definition of <a href="https://en.wikipedia.org/wiki/Carlson_symmetric_form" target="_blank">Carloson's elliptic integrals</a>:
$$
    R_F (x,y,z) = \frac{1}{2} \int^\infty_0 \frac{ds}{(s+x)(s+y)(s+z)}
$$
and, by making the substitution $s+t=u$, we notice that:
$$
    \wp^{-1}(t) = R_F (z-e_1,z-e_2,z-e_3)
$$

</div>

In [10]:
def InverseWeierstrassP(z,g2,g3):
    # Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
    coeffs = [4, 0, -g2, -g3]

    # Finding the roots
    roots = np.roots(coeffs)
    
    # Sorting the roots
    roots = np.sort_complex(roots)
    e1, e2, e3 = roots
    
    return elliprf(z-e1, z-e2, z-e3)

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">1.6. $\wp'$</h2>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Now, using the formula provided by <a href="https://functions.wolfram.com/EllipticFunctions/WeierstrassPPrime/27/02/03/0001/" target="_blank">Wolfram</a>
$$
    \wp'(z) = - \frac{\pi^3}{4\omega_1^3} \frac{\theta_2 \left( \frac{\pi z}{2\omega_1},q \right)\theta_3 \left( \frac{\pi z}{2\omega_1},q \right)\theta_4 \left( \frac{\pi z}{2\omega_1},q \right)\theta'_1 \left( 0,q \right)^3}{\theta_2 \left( 0,q \right)\theta_3 \left( 0,q \right)\theta_4 \left( 0,q \right)\theta_1 \left( \frac{\pi z}{2\omega_1},q \right)^3}
$$

</div>

In [11]:
def WeierstrassPPrime(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    numerator = jtheta(2, pi*z/(2*Omega1(g2,g3)),q)*jtheta(3, pi*z/(2*Omega1(g2,g3)),q)*jtheta(4, pi*z/(2*Omega1(g2,g3)),q)*(jtheta(1, 0, q, derivative=1)**3)
    denominator = jtheta(2, 0,q)*jtheta(3, 0,q)*jtheta(4, 0,q)*(jtheta(1, pi*z/(2*Omega1(g2,g3)),q)**3)
    res = -(1/4)*(pi/Omega1(g2,g3))**3 * (numerator/denominator)
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">2. $\sigma$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

As in the previous section, based on formulas <a href="http://dlmf.nist.gov/23.6.E8" target="_blank">(23.6.8-9),</a> we can write
$$
    \sigma (z) = 2 \omega_1 \exp{\left( \frac{\eta_1 z^2}{2 \omega_1} \right)} \frac{\theta_1(\pi z/(2 \omega_1),q)}{\pi \theta '_1 (0,q)},
$$
where
$$
    \eta_1 = - \frac{\pi^2}{12 \omega_1} \frac{\theta_1 ''' (0,q)}{\theta_1'(0,q)}
$$

</div>

In [12]:
def eta1(g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    res = - (pi**2/(12*Omega1(g2,g3))) * (jtheta(1, 0, q, derivative=3)/jtheta(1, 0, q, derivative=1))
    return res

In [13]:
def WeierstrassSigma(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    Exp = exp((eta1(g2, g3) * z**2) / (2 * Omega1(g2, g3)))
    res = 2*Omega1(g2,g3)*Exp * (jtheta(1, pi * z/(2*Omega1(g2,g3)),q)/(pi*jtheta(1, 0, q, derivative=1)))
    return res

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">3. $\zeta$-function</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

Here, also based on the basic property of  $\zeta$ function <a href="http://dlmf.nist.gov/23.2.E8" target="_blank">(23.2.8)</a>, we can write
$$
    \zeta (z) = \frac{\sigma'(z)}{\sigma(z)} = \frac{\eta_1}{\omega_1} z + \frac{\theta'_1(\pi z/(2 \omega_1),q)}{\theta_1(\pi z/(2 \omega_1),q)} .
$$
</div>

In [14]:
def WeierstrassZeta(z,g2,g3):
    q = qfrom(tau=Omega3(g2,g3)/Omega1(g2,g3))
    firstP = eta1(g2,g3) * z/Omega1(g2,g3)
    secondP =  pi/(2*Omega1(g2,g3)) *jtheta(1,  pi * z/(2*Omega1(g2,g3)), q, derivative=1)/jtheta(1,  pi * z/(2*Omega1(g2,g3)), q)
    return firstP + secondP

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h1 style="font-size: 32px;">4. Tests</h1>
</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">

As a test of the definitions presented above, we will check whether they satisfy the classical formulas presented in the <a href="https://dlmf.nist.gov/23" target="_blank">DLMF</a>  database

</div>

<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.1. Reduce argument</h2>
</div>

In [58]:
# All real roots

# We choose invariants
G2 = 28
G3 = -4

# Argument
zz = 10.2321

# Reduced argument
rz=reduce_argument(zz,G2,G3)

# Rregular value
wp = WeierstrassP(zz,G2,G3)

# After reduction 
wpr = WeierstrassP(rz,G2,G3)

print("Test:", wp- wpr)


Test: (-1.054655281005110285461046e-23 + 6.432315947499379217068928e-38j)


In [16]:
# One complex root

# We choose invariants
G2 = - 24.4439 - 66.1409j
G3 = 80.8878 + 132.282j

# Argument
zz = 10.2321

# Reduced argument
rz=reduce_argument(zz,G2,G3)

# Rregular value
wp = WeierstrassP(zz,G2,G3)

# After reduction 
wpr = WeierstrassP(rz,G2,G3)

print("Test:", wp- wpr)

Test: (8.271806125530276748714087e-25 - 4.135903062765138374357043e-25j)


In [59]:
# Two complex roots

# We choose invariants
G2 = 1.234
G3 = -4.213

# Argument
zz = 10.2321

# Reduced argument
rz=reduce_argument(zz,G2,G3)

# Rregular value
wp = WeierstrassP(zz,G2,G3)

# After reduction 
wpr = WeierstrassP(rz,G2,G3)

print("Test:", wp- wpr)

Test: (-3.877409121342317225959728e-26 + 1.981712768190875745340692e-25j)


In [60]:
# Three complex roots

# We choose invariants
G2 = -(36.1959 + 26.5329j)
G3 = -(53.0658 - 40.3918j)

# Argument
zz = 10.2321

# Reduced argument
rz=reduce_argument(zz,G2,G3)

# Rregular value
wp = WeierstrassP(zz,G2,G3)

# After reduction 
wpr = WeierstrassP(rz,G2,G3)

print("Test:", wp- wpr)

Test: (2.584939414228211483973152e-25 + 1.809457589959748038781207e-25j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.2. Inverse function</h2>
</div>

In [62]:
# All real roots

# We choose invariants
G2 = 28
G3 = -4

# Argument
zz = 10.2321

wp = WeierstrassP(zz,G2,G3)
print("Test:", zz- InverseWeierstrassP(wp,G2,G3))

Test: (9.865660751359038861283672 + 1.623379729270097899916687e-39j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
We can see that there is a big difference, but in reality everything works. We can show it like this:
</div>

In [63]:
G2 = 28
G3 = -4

# Argument
zz = 10.2321

wp = WeierstrassP(zz,G2,G3)

zInv= InverseWeierstrassP(wp,G2,G3)

wpInv = WeierstrassP(zInv,G2,G3)

print("Test:", wp- wpInv)

Test: (-1.645657463196698350559716e-16 + 4.621076195850171932786951e-55j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
In general, arbitrary comparisons of the results of the inverse function do not make sense because $\wp$ is periodic so we have many different arguments $z$ for which we get the same value. Therefore we should compare values after reduction to the fundamental region.
</div>

In [65]:
# We choose invariants
G2 = 28
G3 = -4
# Argument
zz = 10.2321
# Reduced argument
rz=reduce_argument(zz,G2,G3)

wp = WeierstrassP(zz,G2,G3)
# Inverted argument
zinv=InverseWeierstrassP(wp,G2,G3)
# Reduced argument
rzinv=reduce_argument(zinv,G2,G3)

print("Test:", rz- rzinv)

Test: (4.153289139083038579829742e-18 + 1.623379729270097899916687e-39j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
However, the best test is still to check if the $\wp$ function returns the same, so we'll stick to that test
</div>

In [66]:
#One complex root

# We choose invariants
G2 = - 24.4439 - 66.1409j
G3 = 80.8878 + 132.282j

# Argument
zz = 10.2321

wp = WeierstrassP(zz,G2,G3)

zInv= InverseWeierstrassP(wp,G2,G3)

wpInv = WeierstrassP(zInv,G2,G3)

print("Test:", wp- wpInv)

Test: (-5.869893053394364205384472e-16 - 2.392497561220918219515873e-15j)


In [67]:
# Two complex roots

# We choose invariants
G2 = 1.234
G3 = -4.213

# Argument

zz = 10.2321
wp = WeierstrassP(zz,G2,G3)

zInv= InverseWeierstrassP(wp,G2,G3)

wpInv = WeierstrassP(zInv,G2,G3)

print("Test:", wp- wpInv)

Test: (1.943754486420073541485181e-16 + 3.411773133107818931284607e-17j)


In [68]:
# Three complex roots

# We choose invariants
G2 = -(36.1959 + 26.5329j)
G3 = -(53.0658 - 40.3918j)


# Argument
zz = 10.2321

wp = WeierstrassP(zz,G2,G3)

zInv= InverseWeierstrassP(wp,G2,G3)

wpInv = WeierstrassP(zInv,G2,G3)

print("Test:", wp- wpInv)

Test: (2.960235646336669733874125e-16 + 1.184148644083130212440064e-15j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.3. Periodicity</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.3.1.</h4>
    <p> For any half-periods $\omega_j$ and $\omega_k$, where $j=1,2,3$, $k=1,2,3$ and $j\neq k$ we have (Eq.  <a href="http://dlmf.nist.gov/23.2.E9" target="_blank">(23.2.9)</a>):
$$
    \wp(z+2m\omega_j + 2n\omega_k) = \wp(z)
$$
   <p>
   </div>   

In [29]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz= 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

# Tests for 
n=1
m=1
test1 = WeierstrassP(zz + 2*m*w1 + 2*n*w2,G2,G3)-WeierstrassP(zz,G2,G3)
test2 = WeierstrassP(zz + 2*m*w1 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)
test3 = WeierstrassP(zz + 2*m*w2 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)

print("test1=",test1)
print("test2=",test2)
print("test2=",test3)

# Tests for 
n=10
m=-7
test4 = WeierstrassP(zz + 2*m*w1 + 2*n*w2,G2,G3)-WeierstrassP(zz,G2,G3)
test5 = WeierstrassP(zz + 2*m*w1 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)
test6 = WeierstrassP(zz + 2*m*w2 + 2*n*w3,G2,G3)-WeierstrassP(zz,G2,G3)

print("test4=",test4)
print("test5=",test5)
print("test6=",test6)

test1= (-1.033975765691284593589261e-25 + 1.292469707114105741986576e-26j)
test2= (2.584939414228211483973152e-25 - 2.584939414228211483973152e-25j)
test2= (1.033975765691284593589261e-25 + 7.754818242684634451919456e-26j)
test4= (3.205324873642982240126709e-24 + 1.938704560671158612979864e-24j)
test5= (1.447566071967798431024965e-24 - 6.203854594147707561535565e-25j)
test6= (2.429843049374518794934763e-24 - 1.641436528034914292322952e-24j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.3.2.</h4>
    <p> For any half-period $\omega_j$, where $j=1,2,3$, we have (Eq. <a href="http://dlmf.nist.gov/23.2.E10" target="_blank">(23.2.10)</a>):
$$
    \wp'(\omega_j) = 0
$$
   <p>
   </div>   

In [30]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Pp(w1)=", WeierstrassPPrime(w1,G2,G3))
print("Pp(w2)=", WeierstrassPPrime(w2,G2,G3))
print("Pp(w2)=", WeierstrassPPrime(w2,G2,G3))

Pp(w1)= (-6.931760603104882576931517e-43 - 6.296721374115339483675423e-26j)
Pp(w2)= (1.22721438984159158065958e-25 + 1.316070721200402623363533e-25j)
Pp(w2)= (1.22721438984159158065958e-25 + 1.316070721200402623363533e-25j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.3.3.</h4>
    <p> The function $\zeta(z)$ is quasi-periodic and for $j=1,2,3$ satisfies (Eq.  <a href="http://dlmf.nist.gov/23.2.E11" target="_blank">(23.2.11)</a>):
    $$
        \zeta(z+2\omega_j) = \zeta(z) + 2\zeta(\omega_j)
    $$
   <p>
   </div>   

In [31]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Test for w1 =", WeierstrassZeta(zz+2*w1,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w1,G2,G3))
print("Test for w2 =", WeierstrassZeta(zz+2*w2,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w2,G2,G3))
print("Test for w3 =", WeierstrassZeta(zz+2*w3,G2,G3)-WeierstrassZeta(zz,G2,G3)-2*WeierstrassZeta(w3,G2,G3))

Test for w1 = (4.591554204828980849167322e-27 + 5.169878828456422967946304e-26j)
Test for w2 = (-2.584939414228211483973152e-26 + 0.0j)
Test for w3 = (-7.754818242684634451919456e-26 + 1.292469707114105741986576e-26j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.3.4.</h4>
    <p> The function $\sigma(z)$ is quasi-periodic and for $j=1,2,3$ satisfies (Eq.  <a href="http://dlmf.nist.gov/23.2.E15" target="_blank">(23.2.15)</a>):
    $$
        \sigma(z+2\omega_j) = - e^{2\eta_j (z+\omega_j)}\sigma(z), \quad \eta_j = \zeta(\omega_j)
    $$
   <p>
   </div>   

In [32]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

print("Test for w1 =", WeierstrassSigma(zz+2*w1,G2,G3)+exp(2*WeierstrassZeta(w1,G2,G3)*(zz+w1))* WeierstrassSigma(zz,G2,G3))
print("Test for w2 =", WeierstrassSigma(zz+2*w2,G2,G3)+exp(2*WeierstrassZeta(w2,G2,G3)*(zz+w2))* WeierstrassSigma(zz,G2,G3))
print("Test for w3 =", WeierstrassSigma(zz+2*w3,G2,G3)+exp(2*WeierstrassZeta(w3,G2,G3)*(zz+w3))* WeierstrassSigma(zz,G2,G3))

Test for w1 = (-2.150669592637871954665663e-23 - 9.926167350636332098456904e-24j)
Test for w2 = (-3.437969420923521273684292e-24 + 0.0j)
Test for w3 = (-6.352747104407252543012419e-22 + 1.482307657695025593369564e-21j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.4. Differential equation</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.4.1.</h4>
    <p> 
        Then the $\wp$-function satisfies the differential equation (Eq. <a href="http://dlmf.nist.gov/23.3.E10" target="_blank">(23.3.10)</a>)
    $$
        \wp'^2(z) = 4 \wp^3(z) - g_2 \wp(z) -g_3
    $$ 
   <p>
   </div>  

In [33]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

left = WeierstrassPPrime(zz,G2,G3)**2
right = 4*WeierstrassP(zz,G2,G3)**3 - G2*WeierstrassP(zz,G2,G3) - G3

print("Test=", left - right)

Test= (-1.614271245345283503057947e-15 + 2.562300535517916121622207e-16j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.5. Roots</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.5.1.</h4>
    <p> 
    Given any half-perods $\omega_j$, $j=1,2,3$ we can identify the $e_j$ individually, via (Eq. <a href="http://dlmf.nist.gov/23.3.E9" target="_blank">23.3.9</a>)
    $$
        e_j = \wp(\omega_j)
    $$
   <p>
   </div>  

In [34]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
zz = 3.213 - j*0.213

# Half-periods
w1 = Omega1(G2,G3)
w2 = Omega2(G2,G3)
w3 = Omega3(G2,G3)

# Coefficients of the polynomial 4*x**3 - g2*x - g3 = 0
coeffs = [4, 0, -G2, -G3]

# Finding the roots
roots = np.roots(coeffs)
    
# Sorting the roots
roots = np.sort_complex(roots)
e1, e2, e3 = roots

print("e1 - wp(w1)=", e1 - WeierstrassP(w1,G2,G3) )
print("e2 - wp(w2)=", e2 - WeierstrassP(w2,G2,G3) )
print("e3 - wp(w3)=", e3 - WeierstrassP(w3,G2,G3) )

e1 - wp(w1)= (1.324551388116140543331598e-16 - 3.49091863489540942883054e-18j)
e2 - wp(w2)= (1.777149086210981423180532e-16 + 6.981837347299436801338255e-18j)
e3 - wp(w3)= (1.324551388116140543331598e-16 - 3.490918686574415471810185e-18j)


<div style="font-family: 'Latin Modern Roman', Times, serif; font-size: 16px;">
<h2 style="font-size: 24px;">4.6. Addition formulas</h2>
</div>

<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.6.1.</h4>
    <p>
    Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E1" target="_blank">23.10.1</a>):      
    $$
        \wp(u+v) = \frac{1}{4} \left( \frac{\wp'(u) - \wp'(v)}{\wp(u) - \wp(v)}\right)^2 - \wp(u) - \wp(v)
    $$ 
   <p>
   </div>  

In [35]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassP(u + v,G2,G3)
right = (1/4)*((WeierstrassPPrime(u,G2,G3)-WeierstrassPPrime(v,G2,G3))/(WeierstrassP(u,G2,G3)-WeierstrassP(v,G2,G3)))**2 - WeierstrassP(u,G2,G3) - WeierstrassP(v,G2,G3)
print("Test=", left-right)

Test= (2.235086292545507092537055e-16 - 4.717514430966485958251003e-25j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.6.2.</h4>
    <p> 
       Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E2" target="_blank">23.10.2</a>):      
    $$
        \zeta(u+v) = \zeta(u) + \zeta(v) + \frac{1}{2} \frac{\zeta''(u)-\zeta''(v)}{\zeta'(u) - \zeta'(u)}
    $$ 
        Hovewer we have Eq. <a href="http://dlmf.nist.gov/23.2.E7" target="_blank">23.2.7</a> $\wp(z) = - \zeta'(z)$, so
    $$
        \zeta(u+v) = \zeta(u) + \zeta(v) + \frac{1}{2} \frac{\wp'(u)-\wp'(v)}{\wp(u) - \wp(u)}
    $$     
   <p>
   </div>  

In [36]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassZeta(u + v,G2,G3)
right = WeierstrassZeta(u ,G2,G3) +WeierstrassZeta(v,G2,G3) + (1/2) *(WeierstrassPPrime(u,G2,G3)-WeierstrassPPrime(v,G2,G3))/(WeierstrassP(u,G2,G3)-WeierstrassP(v,G2,G3))
print("Test=", left-right)

Test= (-2.067951531382569187178522e-25 - 3.877409121342317225959728e-26j)


<div style="background-color:  rgba(224, 224, 224, 0.25); 
    font-family: 'Latin Modern Roman', Times, serif; font-size: 16px; border: 1px solid  rgb(192, 192, 192); padding: 15px; border-radius: 5px; ">
    <h4 style="font-size: 20px;color: rgb(160, 160, 160);">Definition 4.6.3.</h4>
    <p> 
        Let $u,v\in\mathbb{C}$, so that $u,v,u+v,u-v\notin\Lambda$. Then one has (Eq. <a href="http://dlmf.nist.gov/23.10.E3" target="_blank">23.10.3</a>):      
    $$
        \frac{\sigma(u+v)\sigma(u-v)}{\sigma^2(u)\sigma^2(v)} = \wp(v) - \wp(u)
    $$ 
   <p>
   </div>  

In [37]:
# We choose invariants
G2 = 1.895312
G3 = -7.126521

# Argument
u = 3.213 - j*0.213
v = 1.986 + j*1.218

left = WeierstrassSigma(u + v,G2,G3)* WeierstrassSigma(u - v,G2,G3)/( WeierstrassSigma(u,G2,G3)**2 *  WeierstrassSigma(v,G2,G3)**2)
right = WeierstrassP(v,G2,G3)-WeierstrassP(u,G2,G3)
print("Test=", left-right)

Test= (1.075334796318935977332831e-23 - 3.101927297073853780767783e-24j)
